## Recommendation System

In [ ]:
# Import the libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings 
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid',palette='viridis')
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
# Load the dataset

In [4]:
df=pd.read_csv(r'C:\Users\sanket\Desktop\DS ASSIGNMENT\Recommendation System\Recommendation System\anime.csv')
df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [5]:
df.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [ ]:
# check for null values

In [7]:
df.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [ ]:
#drop null values

In [8]:
df=df.dropna()

In [9]:
df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


### Feature Extraction

In [ ]:
# Handle genres - assuming genres are in a list format, split genres into individual items

In [10]:
df['genre'] = df['genre'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df['genre'])

In [ ]:
# Convert to DataFrame and merge back

In [11]:
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)
df= pd.concat([df, genre_df], axis=1)
df

,anime_id,name,genre,type,episodes,rating,members,Adventure,Cars,Comedy,...,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi
0,32281.0,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114.0,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",TV,64,9.26,793665.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977.0,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",TV,51,9.25,114262.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253.0,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,9.17,673572.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969.0,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",TV,51,9.16,151266.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Normalize ratings 

In [13]:
scaler = MinMaxScaler()
df['rating'] = scaler.fit_transform(df[['rating']])

In [14]:
df

,anime_id,name,genre,type,episodes,rating,members,Adventure,Cars,Comedy,...,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi
0,32281.0,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,0.924370,200630.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114.0,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",TV,64,0.911164,793665.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977.0,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",TV,51,0.909964,114262.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253.0,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,0.900360,673572.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969.0,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",TV,51,0.899160,151266.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
df.isnull().sum()

anime_id        9
name            9
genre           9
type            9
episodes        9
               ..
Super Power     9
Supernatural    9
Thriller        9
Vampire         9
Yaoi            9
Length: 89, dtype: int64

In [79]:
# Fill missing values for numerical columns with mean or median
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].mean())

In [80]:
df.isnull().sum()

anime_id        0
name            9
genre           9
type            9
episodes        9
               ..
Super Power     0
Supernatural    0
Thriller        0
Vampire         0
Yaoi            0
Length: 89, dtype: int64

In [81]:
# Fill missing values for categorical columns with mode
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].fillna(0)

In [51]:
df.isnull().sum()

anime_id        0
name            0
genre           0
type            0
episodes        0
               ..
Super Power     0
Supernatural    0
Thriller        0
Vampire         0
Yaoi            0
Length: 89, dtype: int64

In [82]:
df.isnull().sum().sum()

0

###  Recommendation System

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
def recommend_anime(name,df, top_n=5, threshold=0.5):
    features = df.drop(['anime_id', 'name', 'genre', 'type', 'episodes', 'members'], axis=1) # Select features for similarity
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    target_index = df[df['name'] == name].index[0] # Find the index of the target anime
    similarity_scores = list(enumerate(similarity_matrix[target_index])) # Get similarity scores for the target anime
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True) # Sort by similarity score
    recommendations = [(df.iloc[i[0]].name, i[1]) for i in similarity_scores if i[1] >= threshold and i[0] != target_index] # Select anime above similarity threshold and exclude the target anime itself
    return recommendations[:top_n] # Return top N recommendations


In [59]:
# Test the function
print(recommend_anime("Gintama°", df, top_n=5, threshold=0.5))

[(4, 0.9999933158708841), (9, 0.9999838036663923), (8, 0.9999814021366503), (12, 0.9999634879221596), (63, 0.9996460162769004)]


### Evaluation

In [67]:
# Ensure 'genre' column has no NaN values or convert any NaNs to empty strings
df['genre'] = df['genre'].fillna('')

In [68]:
# Convert 'genre' values to lists by splitting on commas
df['genre'] = df['genre'].apply(lambda x: x.split(',') if isinstance(x, str) else [])

In [69]:
# Check that the transformation worked correctly
print(df['genre'].head())

0    []
1    []
2    []
3    []
4    []
Name: genre, dtype: object


In [71]:
from sklearn.preprocessing import MultiLabelBinarizer

In [72]:
# Use MultiLabelBinarizer to one-hot encode genres
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df['genre'])

In [73]:
# Convert encoded genres to a DataFrame and merge back
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)
df= pd.concat([df, genre_df], axis=1)

In [83]:
df

,anime_id,name,genre,type,episodes,rating,members,Adventure,Cars,Comedy,...,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi
0,32281.000000,Kimi no Na wa.,[],Movie,1,0.924370,200630.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5114.000000,Fullmetal Alchemist: Brotherhood,[],TV,64,0.911164,793665.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,28977.000000,Gintama°,[],TV,51,0.909964,114262.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,9253.000000,Steins;Gate,[],TV,24,0.900360,673572.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,9969.000000,Gintama&#039;,[],TV,51,0.899160,151266.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12265,13638.001165,0,0,0,0,0.577223,18348.879005,0.072481,0.003079,0.118832,...,0.001082,0.004078,0.008571,0.000083,0.003495,0.000083,0.001165,0.000083,0.000083,0.000166
12274,13638.001165,0,0,0,0,0.577223,18348.879005,0.072481,0.003079,0.118832,...,0.001082,0.004078,0.008571,0.000083,0.003495,0.000083,0.001165,0.000083,0.000083,0.000166
12279,13638.001165,0,0,0,0,0.577223,18348.879005,0.072481,0.003079,0.118832,...,0.001082,0.004078,0.008571,0.000083,0.003495,0.000083,0.001165,0.000083,0.000083,0.000166
12280,13638.001165,0,0,0,0,0.577223,18348.879005,0.072481,0.003079,0.118832,...,0.001082,0.004078,0.008571,0.000083,0.003495,0.000083,0.001165,0.000083,0.000083,0.000166


In [84]:
# Split dataset
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Evaluate recommendations

In [88]:
def evaluate_recommendation_system(test_data,df,threshold=0.5):
    precision_scores, recall_scores, f1_scores = [], [], []
    for name in test_data['name'].sample(10):  # Sample for faster evaluation
        recommended_anime = recommend_anime(name, df, threshold=threshold)
        true_genres = set(test_data[test_data['name'] == name['genre'].values[0]])  # Assuming ground truth is the genres of the anime in the test set
        recommended_genres = set([anime_data[anime_data['name'] == rec[0]]['genre'].values[0] for rec in recommended_anime])
        if recommended_genres:            # Calculate precision, recall, and F1
            precision = precision_score(list(true_genres), list(recommended_genres), average='binary')
            recall = recall_score(list(true_genres), list(recommended_genres), average='binary')
            f1 = f1_score(list(true_genres), list(recommended_genres), average='binary')
            precision_scores.append(precision)
            recall_scores.append(recall)
            f1_scores.append(f1)

    # Average scores
    avg_precision = sum(precision_scores) / len(precision_scores)
    avg_recall = sum(recall_scores) / len(recall_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)
    return avg_precision, avg_recall, avg_f1

In [ ]:
precision, recall, f1 = evaluate_recommendation_system(test_data,df, threshold=0.7)
print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")